# Programming for Data Analysis Assigment

## Initial file creation, assignment to be specified at a later date

"For this project you must create a dataset by simulating a real-world phenomenon of your chooising. You may pick any phenomenon you wish - you might pick on that is of interest to you in your personal or professional life. Then, rather than collect data related to the phenomenon, you should model and synthesise such data using Python. We suggest you use the `numpy.random` package for this purpose.

Specificailly, in this project you should:

1. Choose a real-world phenomenon that can be measured and for which you could collect at least one-hundred data points across at least four different variables;

2. Investigate the types of variables involved, their likely distributions, and their relationships with each other;

3. Synthesise/simulate a dataset as closely matching their properties as possible;

4. Detail your research and implement the simulation in a Jupyter notebook - the dataset itself can simply be display in an output cell within the notebook.

Note that this project is about simulation - you must synthesise a dataset. Some students may already have some real-world datasets in their own files. It is okay to base your synthesised dataset on these should you wish (please reference it if you do), but the main task is in this project is to create a synthesised dataset."

For point two, it is not enough that you end up with a totally random dataset, you need to see which distribution might be most appropriate for that particular item. What are the likely data points that you collect from a phenomenon? How are the variables related to each other? What does the distribution look like? Is it a normal distribution? A Poisson distribution?

It is sufficient for the dataset to be a pandas dataframe, it doesn't need to be a CSV or XML file.

Brian: "So I'm not actually that interested in the actual data. What we're most interested in is the research investigations you do to create the dataset, because presumably there will be some random element in the generation of your data, following some sort of distribution. And I'll be able to re-run your notebook and get a different randomly-generated dataset with the same properties."

## Basic Plan

So you need to find a basic dataset that can be extrapolated, rather than something comprehensive. The starting points should probably be either Hitting Against the Spin or the Vollman hockey book. This could be something team-based or player-based, something that can be condensed into small enough chunks that it fits into a 100 row by 5 column (for the four variables) grid.

Remember that Brian's example was that he thinks there's a correlation between a student's grade and their level of degree, the amount of time spent studying and the number of commits they make.

Could even be the amount of wins gained from winning the toss? Variables would be: toss result, choice of batting/fielding first, match result, total runs in the match. There's ten test teams so the last ten or fifteen results could be handy. Could be: there's a lot of noise about Austin Matthews' 5-on-5 play vs. his shots and likewise his PP goals/shots, how will he do against teams if we can predict their 5-on-5 play and their PK? Variables could be: SOG EV, SOG PP, Goals EV, Goals PP, Opp PK % (for quickness I'd use their overall [season performance](https://www.nhl.com/stats/teams?reportType=season&seasonFrom=20212022&seasonTo=20222023&gameType=2&filter=gamesPlayed,gte,1&sort=points,wins&page=0&pageSize=50), rather than their season-to-date performance to that point, TOI, etc., his 2021/2022 season stats are [here](https://www.nhl.com/player/auston-matthews-8479318?stats=gamelogs-r-nhl&season=20212022).

Vollman definitely tell you where he sources all of his data from, and I'm guessing it will be either NHL.com or hockey-reference.com. HAIS the spin should just be a matter of going to Cricinfo and seeing what we can dig out from their core metrics.

After this you'll need to show investigation of each variable, and plot them accordingly using NumPy or Seaborn to see if a distribution can be identified. Once these are identified, even just using [Wikipedia](https://en.wikipedia.org/wiki/Probability_distribution#External_links) or the [NumPy documentation](https://numpy.org/doc/stable/reference/random/generator.html), you'll need to use the latest notebooks from this module to simulate a dataset accordingly, so if you plot, say, expected goals, and you can see that the data is Poisson-distributed, you can use the `poisson` function in NumPy to generate the data.

In [1]:
# did you use all of these libraries?
import datetime as dt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn as sk